# Building a research Agent

Code basiert auf Auszügen von:



*   https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_3_pro.ipynb#scrollTo=3CaXL22k8iw4
*   https://www.kaggle.com/code/kaggle5daysofai/day-1b-agent-architectures



In [ ]:
# Wir installieren die Google GenAI libraries
%pip install --upgrade --quiet google-genai
%pip install google-adk

## Importieren der Packages

In [ ]:
import os
import sys

from IPython.display import HTML, Markdown, display
from google import genai
from google.genai import types
from pydantic import BaseModel

In [ ]:
#import sys
#
#if "google.colab" in sys.modules:
#    from google.colab import auth
#    auth.authenticate_user()

In [ ]:
from google import genai
from google.colab import userdata

# Definieren der notwendigen Umgebungsvariablen.
# ACHTUNG: Sie benötigen einen Google API Key

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
client = genai.Client(api_key=GEMINI_API_KEY)

In [ ]:
print(f"Google API Key set: {'Yes' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'No (REPLACE PLACEHOLDER!)'}")

Google API Key set: Yes


In [ ]:
# Test, ob Aufruf des Modells alleine funktioniert.
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how Diffusion models work in a few words"
)
print(response.text)

They learn to progressively **denoise** data. To generate, they iteratively remove noise from pure random static until a clear sample emerges.


In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [ ]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

## Definieren der Agenten

### Research Agent

In [ ]:
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        api_key=GEMINI_API_KEY,
        retry_options=retry_config
    ),
    instruction="""You are a world class specialized research agent.
    Your only job is to use the Google_search tool to find 2-3 pieces of
    relevant information on the given topic and present the findings with
    citations.""",
    tools=[google_search],
    # Results of this agent will be stored in the session state with this name.
    output_key="research_findings",
)

### Summarizer Agent

In [ ]:
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        api_key=GEMINI_API_KEY,
        retry_options=retry_config
    ),
    # instruction modified to request a bulleted list for a clear output format.
    instruction="""Read the provided research findings: {research_findings}.
    Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

### Main Agent

In [ ]:
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        api_key=GEMINI_API_KEY,
        retry_options=retry_config
    ),
    # instruction tells root agent HOW to use its tools (which are the other agents).
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
    1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
    2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
    3. Finally, present the final summary clearly to the user as your response.""",
    # wrap sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

In [ ]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest advancements in Diffusion models and what do they mean for AI?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in Diffusion models and what do they mean for AI?
ResearchCoordinator > Latest advancements in diffusion models are significantly enhancing their capabilities in generating high-quality and realistic content, including images, videos, and audio. These models work by iteratively refining random noise into structured data, a process inspired by natural diffusion phenomena.

Key advancements and their implications include:

*   **Improved Generative Quality and Control:** Diffusion models now achieve state-of-the-art performance in image and audio generation, often surpassing traditional models like Generative Adversarial Networks (GANs). Models like Gen-1 and Gen-2 demonstrate remarkable progress in text-to-video synthesis, producing videos with enhanced detail and control. The development of conditional diffusion models allows for guided generation based on specific criteria.
*   **Enhanced Efficiency a

In [ ]:
print(response)

[Event(model_version='gemini-2.5-flash-lite', content=Content(
  parts=[
    Part(
      function_call=FunctionCall(
        args={
          'request': 'latest advancements in Diffusion models and their implications for AI'
        },
        id='adk-f4564cd5-53ec-42bc-bc54-bf1128a6f0a8',
        name='ResearchAgent'
      )
    ),
  ],
  role='model'
), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
  candidates_token_count=23,
  prompt_token_count=190,
  prompt_tokens_details=[
    ModalityTokenCount(
      modality=<MediaModality.TEXT: 'TEXT'>,
      token_count=190
    ),
  ],
  total_token_count=213
), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, cache_metadata=None, citation_metadata=None, invocation_id='e-0f65eb